# Google Colab file for paraphrasing the MultiNLI development set. 
#### this version uses a large BART seq2seq (text2text generation) model fine-tuned on 3 paraphrase datasets.

https://huggingface.co/eugenesiow/bart-paraphrase

In [ ]:
# install libraries
!pip install SentencePiece 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.8 MB/s eta 0:00:00


In [ ]:
# import libraries
import torch
import json
from datetime import datetime
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
# set seed and torch device
# Colab notebook setting "runtime type" must be set to GPU
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
device = "cuda"

In [ ]:
# get the model
model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase').to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [ ]:
# define functions
def paraphrase_sentence(sentence):
  # tokenize the sentence and run through the BART model
  batch = tokenizer(sentence, return_tensors='pt').to(device)
  generated_ids = model.generate(batch['input_ids'], max_length=128)
  return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

def paraphrase_dataset(path_original, path_out):
  # open files
  with open(path_out, "w") as out_f:
    with open(path_original, "r") as f:
      for id_, row in enumerate(f):
        data = json.loads(row)
        # paraphrase the sentences
        new_data = {"label": data["label"],
                    "premise": paraphrase_sentence(data["premise"]),
                    "hypothesis": paraphrase_sentence(data["hypothesis"])}

        # checking progress
        if id_ % 400 == 0:
          print(id_, datetime.now())

        # export the sentences
        json.dump(new_data, out_f)
        out_f.write('\n')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read files from Google Drive and export to Google Drive
paraphrase_dataset("/content/drive/MyDrive/thesis/multinli_dev_matched.jsonl", 
                   "/content/drive/MyDrive/thesis/multinli_dev_matched_bart.jsonl")

paraphrase_dataset("/content/drive/MyDrive/thesis/multinli_dev_mismatched.jsonl", 
                   "/content/drive/MyDrive/thesis/multinli_dev_mismatched_bart.jsonl")

0 2023-05-30 13:07:23.367927
400 2023-05-30 13:11:59.434058
800 2023-05-30 13:16:32.527289
1200 2023-05-30 13:20:59.015041
1600 2023-05-30 13:25:30.014289
2000 2023-05-30 13:30:03.424603
2400 2023-05-30 13:34:35.423831
2800 2023-05-30 13:39:03.668556
3200 2023-05-30 13:43:20.697612
3600 2023-05-30 13:47:54.439071
4000 2023-05-30 13:52:07.572423
4400 2023-05-30 13:56:28.263303
4800 2023-05-30 14:00:46.083281
5200 2023-05-30 14:05:09.343819
5600 2023-05-30 14:09:23.700766
6000 2023-05-30 14:13:45.886004
6400 2023-05-30 14:18:06.525594
6800 2023-05-30 14:22:25.314218
7200 2023-05-30 14:26:54.439776
7600 2023-05-30 14:31:35.800136
8000 2023-05-30 14:36:21.398219
8400 2023-05-30 14:41:08.413496
8800 2023-05-30 14:45:37.925339
9200 2023-05-30 14:50:15.880195
9600 2023-05-30 14:54:56.467328
0 2023-05-30 14:59:33.284349
400 2023-05-30 15:04:18.342472
800 2023-05-30 15:08:59.916630
1200 2023-05-30 15:13:29.475931
1600 2023-05-30 15:18:13.925483
2000 2023-05-30 15:23:10.581145
2400 2023-05-30 15